In [5]:
import os
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import mlflow
import mlflow.sklearn


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

### Leemos pipeline configurado con entrenamiento

In [2]:
with open('../artifacts/pipeline_trained.pkl', 'rb') as f:
    pipeline = pickle.load(f)

#pipeline entrenado

### Realizamos predicciones con la data cruda de Test

In [3]:
test_new_data = pd.read_csv('../data/raw/dataset_test.csv')
test_new_data.drop(['Unnamed: 0'], axis=1, inplace=True)
test_new_data.head()

,id_eq,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,K2O,Na2O,A/S,"""CaO"" [CurrentProduct.Dry basis]","""SiO2"" [CurrentProduct.Dry basis]","""Al2O3"" [CurrentProduct.Dry basis]","""Fe2O3"" [CurrentProduct.Dry basis]","""MgO"" [CurrentProduct.Dry basis]","""CaO"" [Rolling.Analysis1.Dry basis]","""SiO2"" [Rolling.Analysis1.Dry basis]","""Al2O3"" [Rolling.Analysis1.Dry basis]","""Fe2O3"" [Rolling.Analysis1.Dry basis]","""MgO"" [Rolling.Analysis1.Dry basis]"
0,Man_CLK461,20.50086,5.148318,2.384319,66.29183,2.869013,1.796478,0.746897,0.388896,0.633161,41.35239,14.42118,3.54883,1.5619,2.79854,0.884096,0.896789,0.252115,0.123681,0.423213
1,Aus_CLK463,20.77880,4.973646,2.392293,66.04227,2.820995,1.645423,0.756532,0.382241,0.691051,41.35239,14.42118,3.54883,1.5619,2.79854,0.884096,0.896789,0.252115,0.123681,0.423213
2,Man_CLK461,20.67252,5.121428,2.385282,66.48771,2.873353,1.591508,0.692148,0.369799,0.669945,41.35239,14.42118,3.54883,1.5619,2.79854,0.884096,0.896789,0.252115,0.123681,0.423213
3,Man_CLK462,20.96639,5.085337,2.441511,66.20022,2.898922,1.494468,0.680264,0.372012,0.708589,41.35239,14.42118,3.54883,1.5619,2.79854,0.884096,0.896789,0.252115,0.123681,0.423213
4,Aus_CLK463,20.86930,5.001517,2.407377,66.49178,2.881150,1.417410,0.638553,0.376244,0.725919,41.35239,14.42118,3.54883,1.5619,2.79854,0.884096,0.896789,0.252115,0.123681,0.423213


In [4]:
predicts_array = pipeline.predict(test_new_data)
predicts = pd.DataFrame(predicts_array, columns= ['C3S', 'C2S', 'C3A', 'C4AF', 'FCAO'])
predicts

,C3S,C2S,C3A,C4AF,FCAO
0,66.971402,1.378794,9.612604,7.328389,2.286859
1,66.671312,3.588036,9.100192,7.279257,1.875246
2,67.282168,2.024470,9.573436,7.315949,2.193486
3,66.731099,5.426604,9.394051,7.452943,1.391822
4,68.120368,3.176422,9.185223,7.358845,1.616155
...,...,...,...,...,...
8781,69.214696,3.958590,9.347816,7.383923,1.126855
8782,69.948242,1.829694,9.259988,7.292943,1.563903
8783,69.183680,2.651243,9.474566,7.438537,1.557099
8784,69.609728,3.175682,9.231913,7.352670,1.222200


In [6]:
# 1. Generar el nombre del archivo con la fecha y hora
current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
filename = f"{current_time}.csv"

In [9]:
# 2. Configuramos nonbre de carpeta
output_dir = "../data/predictions"

In [10]:
# 3. Guardar el archivo CSV en la carpeta
file_path = os.path.join(output_dir, filename)
predicts.to_csv(file_path, index=False)

In [11]:
# configuracion servidor

mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("Predicts_Cement")

2024/12/21 18:18:25 INFO mlflow.tracking.fluent: Experiment with name 'Predicts_Cement' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/806215297795354040', creation_time=1734826705280, experiment_id='806215297795354040', last_update_time=1734826705280, lifecycle_stage='active', name='Predicts_Cement', tags={}>

In [12]:
with mlflow.start_run():
    # Loguear el artefacto
    mlflow.log_artifact(file_path, artifact_path="predictions")

mlflow.end_run()

🏃 View run trusting-grouse-79 at: http://127.0.0.1:8080/#/experiments/806215297795354040/runs/6b4b5f6b05714e8292aa8534cf8506a1
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/806215297795354040
